In [128]:
from dotenv import load_dotenv
import fitz  # PyMuPDF
import pytesseract
import cv2
from pdf2image import convert_from_path
from PIL import Image
import json
import os
import numpy as np
from google import genai
load_dotenv()

True

In [87]:
OPENROUTER_API_KEY = "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"

Configuration

In [ ]:
os.environ["PATH"] += os.pathsep + "/usr/bin" 
pytesseract.pytesseract.tesseract_cmd = "/usr/bin/tesseract"

DATA_DIR = r"C:\Users\Rafid\Desktop\Thesis_Nubedian\Document_anonymiser\data"
OUTPUT_DIR = r"C:\Users\Rafid\Desktop\Thesis_Nubedian\Document_anonymiser\redacted_output"
os.makedirs(OUTPUT_DIR, exist_ok=True)


checking for pdfs

In [53]:
pdf_files = [f for f in os.listdir(DATA_DIR) if f.lower().endswith(".pdf")]

if pdf is readable text (not scanned copy)

In [54]:
def is_native_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    for page in doc:
        if page.get_text("words"):
            return True
    return False

print(is_native_pdf(os.path.join(DATA_DIR, pdf_files[0])))  # True = native, False = scanned


True


getting all the page content as text

In [104]:
import fitz  # PyMuPDF

pdf_path = os.path.join(DATA_DIR, pdf_files[0])

# Open PDF
doc = fitz.open(pdf_path)

# Extract text page by page
for page_number, page in enumerate(doc, start=1):
    text = page.get_text()  # full text of the page
    print(f"--- Page {page_number} ---")
    print(text)


--- Page 1 ---
Ärztlicher Befundbericht - Anlage zum Antrag auf Anschlussrehabilitation
I. Allgemeine Angaben
I.1. Personalien der/des Versicherten:
HANSMEIERHAUSERBAER
Name
29.06.1968
Geburtsdatum
Oliver
Vorname
26012150
Versichertennummer
Geschlecht
männlich
weiblich
divers
I.2. Die Krankenhausbehandlung ggf. einschl. Frühmobilisation und Wundbehandlung
Aufnahmedatum
02.11.2005
voraussichtliches Entlassungsdatum
I.3. Werden derzeit neurologische, geriatrische oder fachübergreifende  
Frührehabilitationsmaßnahmen durchgeführt?
nein
ja
I.4. Die Anschlussrehabilitation soll erfolgen
als Direktverlegung
innerhalb der gesetzlichen Frist von 14 Tage, frühestens am:
außerhalb der gesetzlichen Frist von 14 Tage, frühestens am: 26.12.2024
Medizinische Begründung:
II. Rehabilitationsbedürftigkeit
II.1. Antragsrelevante Funktionsdiagnosen in der Reihenfolge ihrer Bedeutung 
(Codierung konkret und endständig)
Funktionsdiagnosen
ICD-10-GM 1
Ur-
sache2
0
5
0
0
RTHFGHFG
5
1 Die hier angegebenen Dia

In [56]:
import os
import re
import json
from openrouter import OpenRouter

getting personal information list using llm

In [ ]:
def find_pii_openrouter(text_content, model="gpt-4o-mini", base_url=None):
    from openrouter import OpenRouter
    import os, re, json

    client = OpenRouter(
        api_key=OPENROUTER_API_KEY
    )

    # Create prompt
    system_instruction = """
    You are a data anonymisation assistant.
    Identify PII (personal data) in the text.
    Examples: names, phone numbers, emails, addresses, ID numbers.
    Return ONLY a JSON array like this:
    [
        {"text": "John"},
        {"text": "Doe"},
        {"text": "johndoe@email.com"},
        ...
    ]
    Do not include features name like name, date of birth, insurance number, because they are not 
    sensitive information that should not be redacted.
    """

    messages = [
        {"role": "system", "content": system_instruction},
        {"role": "user", "content": text_content}
    ]

    response = client.chat.send(
        model="gpt-4o-mini",
        messages=messages
    )

    raw_content = response.choices[0].message.content
    # Strip code block if present
    cleaned = re.sub(r"^```json\s*|\s*```$", "", raw_content.strip(), flags=re.MULTILINE)
    try:
        pii = json.loads(cleaned)
    except json.JSONDecodeError:
        pii = []

    return pii


In [113]:
pii_list = find_pii_openrouter(text)

In [114]:
pii_list

[{'text': '26012150'},
 {'text': 'HANSMEIERHAUSERBAER'},
 {'text': 'Oliver'},
 {'text': '29.06.1968'}]

In [126]:
import fitz  # PyMuPDF
OUTPUT_DIR = "C:\\Users\\Rafid\\Desktop\\Thesis_Nubedian\\Document_anonymiser\\redacted_output"
def redact_by_text(pdf_path, pii_list):
    doc = fitz.open(pdf_path)

    for page in doc:
        for pii in pii_list:
            text = pii["text"]
            # Search all occurrences of text → get rectangles automatically
            areas = page.search_for(text)
            for rect in areas:
                page.add_redact_annot(rect, fill=(1,1,1))
        page.apply_redactions()

    base = os.path.basename(pdf_path).replace(".pdf", "_redacted.pdf")
    output_path = os.path.join(OUTPUT_DIR, base)

    doc.save(output_path)
    doc.close()

    return output_path

In [127]:
redacted_pdf = redact_by_text(os.path.join(DATA_DIR, pdf_files[0]), pii_list)

Conversion (PDF Page to Image)

In [52]:
from pdf2image import convert_from_path

pages = convert_from_path(os.path.join(DATA_DIR, pdf_files[0]), dpi=300, fmt="png")
for i, page in enumerate(pages):
    page.save(f"pages/page_{i}.png")


PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH?

get the co ordinates


In [52]:
def is_native_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    for page in doc:
        if page.get_text("words"):
            return True
    return False

def extract_native(pdf_path):
    doc = fitz.open(pdf_path)
    pages_data = []
    for page in doc:
        words = page.get_text("words")
        page_data = [{"text": w[4], "bbox": w[:4]} for w in words]
        pages_data.append(page_data)
    return pages_data

def extract_scanned(pdf_path):
    images = convert_from_path(pdf_path, poppler_path='/usr/bin')
    pages_data = []
    for img in images:
        img_cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
        data = pytesseract.image_to_data(img_cv, output_type=pytesseract.Output.DICT)
        page_data = []
        for i in range(len(data["text"])):
            if data["text"][i].strip() != "":
                x, y, w, h = data["left"][i], data["top"][i], data["width"][i], data["height"][i]
                page_data.append({"text": data["text"][i], "bbox": [x, y, x+w, y+h]})
        pages_data.append(page_data)
    return pages_data, images

def analyze_pii(extracted_text):
    joined = "\n".join([item["text"] for page in extracted_text for item in page])
    client = genai.Client()
    system_instruction = """
    You are a data anonymisation assistant.
    Identify PII words or phrases from the text.
    Return ONLY a JSON array of objects: [{"text": "..."}]
    """
    response = client.models.generate_content(
        model="gemini-1.5-flash",
        contents=joined,
        system_instruction=system_instruction
    )
    try:
        return json.loads(response.text)
    except:
        return []

def redact_native(pdf_path, pii_list, extracted_data):
    doc = fitz.open(pdf_path)
    for page_no, page in enumerate(doc):
        page_words = extracted_data[page_no]
        for pii in pii_list:
            for w in page_words:
                if w["text"] == pii["text"]:
                    rect = fitz.Rect(*w["bbox"])
                    page.add_redact_annot(rect, fill=(0,0,0))
        page.apply_redactions()
    output = os.path.join(OUTPUT_DIR, os.path.splitext(os.path.basename(pdf_path))[0] + "_redacted.pdf")
    doc.save(output)
    return output

def redact_scanned(images, extracted_data, pii_list, pdf_name):
    redacted_images = []
    for i, img in enumerate(images):
        img_cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
        page_words = extracted_data[i]
        for pii in pii_list:
            for w in page_words:
                if w["text"] == pii["text"]:
                    x1, y1, x2, y2 = w["bbox"]
                    cv2.rectangle(img_cv, (x1, y1), (x2, y2), (0,0,0), -1)
        redacted_images.append(Image.fromarray(cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)))
    output_path = os.path.join(OUTPUT_DIR, pdf_name + "_redacted.pdf")
    redacted_images[0].save(output_path, save_all=True, append_images=redacted_images[1:])
    return output_path


In [22]:

if not pdf_files:
    print("No PDF files found in 'data/' folder.")
else:
    for pdf in pdf_files:
        input_path = os.path.join(DATA_DIR, pdf)
        print(f"Processing: {input_path}")
        if is_native_pdf(input_path):
            print(" → Detected Native PDF")
            extracted = extract_native(input_path)
            pii = analyze_pii(extracted)
            output = redact_native(input_path, pii, extracted)
        else:
            print(" → Detected Scanned PDF")
            extracted, images = extract_scanned(input_path)
            pii = analyze_pii(extracted)
            output = redact_scanned(images, extracted, pii, os.path.splitext(pdf)[0])
        print(f" ✔ Saved: {output}\n")

Processing: C:\Users\Rafid\Desktop\Thesis_Nubedian\Document_anonymiser\data\Medical Report with Barthel.pdf
 → Detected Native PDF


ValueError: Missing key inputs argument! To use the Google AI API,provide (`api_key`) arguments. To use the Google Cloud API, provide (`vertexai`, `project` & `location`) arguments.

In [ ]:
# ---------------------------
# Process PDFs in data folder
# ---------------------------
pdf_files = [f for f in os.listdir(DATA_DIR) if f.lower().endswith(".pdf")]

if not pdf_files:
    print("No PDF files found in 'data/' folder.")
else:
    for pdf in pdf_files:
        input_path = os.path.join(DATA_DIR, pdf)
        print(f"Processing: {input_path}")
        if is_native_pdf(input_path):
            print(" → Detected Native PDF")
            extracted = extract_native(input_path)
            pii = analyze_pii(extracted)
            output = redact_native(input_path, pii, extracted)
        else:
            print(" → Detected Scanned PDF")
            extracted, images = extract_scanned(input_path)
            pii = analyze_pii(extracted)
            output = redact_scanned(images, extracted, pii, os.path.splitext(pdf)[0])
        print(f" ✔ Saved: {output}\n")